In [ ]:
# Nos conectamos a nuestro conjunto de datos

from google.colab import drive
drive.mount('/content/drive')

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# optimizador de la red por medio de Bayessian Optimization
#  !pip install bayesian-optimization
from bayes_opt import BayesianOptimization


In [9]:
# Abrimos nuestros dataset para prepararlo
df_train = pd.read_csv('train_data.csv', sep=';', decimal='.')
print(df_train.shape)

df_test = pd.read_csv('test_data.csv', sep=';', decimal='.')
print(df_test.shape)

# Vamos a abrir las imágenes
trainImagesX = np.load("trainImagesX.npy")
print(trainImagesX.shape)

# Abrimos las imagenes de test
testImagesX = np.load("testImagesX.npy")
print(testImagesX.shape)


(750, 22)
(250, 22)
(750, 224, 224, 3)
(250, 224, 224, 3)


In [14]:
for image in trainImagesX:
    if image.shape >= (224, 224, 4):
        print(image.shape)
        plt.imshow(image)
        plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Escalamos las imagenes
scaler = MinMaxScaler()
x_train_images = scaler.fit_transform(trainImagesX)
x_test_images = scaler.fit_transform(testImagesX)

In [ ]:
# Hacemos el split de train y test para las imágenes y los atributos
# Lo hacemos con el dataframe completo, aún no separamos los valores

split = train_test_split(df_train, x_train_images, test_size=0.1, random_state=42)
(trainAttrX, valAttrX, trainImagesX, valImagesX) = split

In [ ]:
# Separamos x_train, y_train y convertimos a numpy
x_train = trainAttrX.drop(['Price'], axis=1).values
y_train = trainAttrX['Price'].values

# Separamos x_test, y_test y convertimos a numpy
x_test = df_test.drop(['Price'], axis=1).values
y_test = df_test['Price'].values

In [ ]:
# Create the scaler object
scaler = MinMaxScaler()

# Fit the scaler to the training data
scaler.fit(x_train)

# Transform the training and test data
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Flatten, Input, Dropout, Conv2D, Activation, BatchNormalization, MaxPooling2D
from tensorflow.keras import optimizers, Model
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
import locale
from matplotlib import image as mpimg
import matplotlib.gridspec as gridspec
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import plot_model



def train_deep_net(width, height, depth, filters=(16, 32, 64), regress=False):
    # initialize the input shape and channel dimension, assuming
    # TensorFlow/channels-last ordering
    inputShape = (height, width, depth)
    chanDim = -1

    # define the model input
    inputs = Input(shape=inputShape)

    # loop over the number of filters
    for (i, f) in enumerate(filters):
        # if this is the first CONV layer then set the input
        # appropriately
        if i == 0:
            x = inputs

        # CONV => RELU => BN => POOL
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

    # flatten the volume, then FC => RELU => BN => DROPOUT
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.5)(x)

    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(4)(x)
    x = Activation("relu")(x)

    # check to see if the regression node should be added
    if regress:
        x = Dense(1, activation="linear")(x)

    # construct the CNN
    model = Model(inputs, x)

    # return the CNN
    return model


In [ ]:
# initialize the number of epochs to train for, initial learning rate,
# and batch size
EPOCHS = 100
INIT_LR = 1e-3
BS = 32

model = train_deep_net(width=64, height=64, depth=3, regress=True)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt, metrics=["mae"])



In [ ]:
# train the model
print("[INFO] training model...")
history = model.fit(x_train_images, y_train,
                    validation_data=(x_test_images, y_test),
                    epochs=EPOCHS,
                    batch_size=BS)



In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x_test_images)
diff = predictions.flatten() - y_test
percentDiff = (diff / y_test) * 100
absPercentDiff = np.abs(percentDiff)
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
	locale.currency(df_train["price"].mean(), grouping=True),
	locale.currency(df_train["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

